In [ ]:
# 練習15-3 尋找輪廓 & 分別繪製輪廓（
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib 字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 讀取圖片
img = cv2.imread('C:\\img\\contours.bmp')

if img is None:
    print("❌ 無法讀取 contours.bmp，請確認檔案路徑")
else:
    print(f"圖片尺寸: {img.shape}")
    
    # 轉換為灰階
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 二值化
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 尋找輪廓（使用 RETR_TREE 取得完整的階層結構）
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"\n找到 {len(contours)} 個輪廓")
    print("="*80)
    
    # 過濾出三個主要物件（最外層且面積最大的輪廓）
    main_contours = []
    
    for i, (contour, hier) in enumerate(zip(contours, hierarchy[0])):
        parent_idx = hier[3]
        if parent_idx == -1:  # 最外層輪廓
            area = cv2.contourArea(contour)
            main_contours.append((i, contour, area))
    
    # 按面積排序，取前3大
    main_contours.sort(key=lambda x: x[2], reverse=True)
    main_contours = main_contours[:3]  # 只取前3個
    
    print(f"\n選取的3個主要物件輪廓:")
    for idx, (i, contour, area) in enumerate(main_contours):
        perimeter = cv2.arcLength(contour, True)
        print(f"  物件 {idx+1}: 輪廓索引={i}, 面積={area:.2f}, 周長={perimeter:.2f}")
    
    print("\n" + "="*80)
    
    # 視覺化 - 2行3列
    fig = plt.figure(figsize=(18, 12))
    
    # 第一行：原始圖、二值圖、所有輪廓
    plt.subplot(2, 3, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Original Image', fontsize=14)
    plt.axis('off')
    
    plt.subplot(2, 3, 2)
    plt.imshow(binary, cmap='gray')
    plt.title('Binary Image', fontsize=14)
    plt.axis('off')
    
    img_all_contours = img.copy()
    cv2.drawContours(img_all_contours, contours, -1, (0, 255, 0), 2)
    plt.subplot(2, 3, 3)
    plt.imshow(cv2.cvtColor(img_all_contours, cv2.COLOR_BGR2RGB))
    plt.title(f'All Contours ({len(contours)})', fontsize=14)
    plt.axis('off')
    
    # 第二行：三個主要物件的輪廓（黑色背景）
    colors = [
        (255, 0, 0),    # 紅色
        (0, 255, 0),    # 綠色
        (0, 0, 255),    # 藍色
    ]
    
    for idx, (i, contour, area) in enumerate(main_contours):
        # 建立黑色背景
        img_single = np.zeros_like(img)
        
        # 在黑色背景上繪製當前輪廓
        cv2.drawContours(img_single, contours, i, colors[idx], 3)
        
        plt.subplot(2, 3, idx + 4)
        plt.imshow(cv2.cvtColor(img_single, cv2.COLOR_BGR2RGB))
        
        perimeter = cv2.arcLength(contour, True)
        plt.title(f'Object {idx+1} (Contour {i})\nArea={area:.0f}, Perimeter={perimeter:.1f}', fontsize=12)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    


In [ ]:
# 練習15-4 輪廓遮罩
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib 字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 讀取圖片
img = cv2.imread('C:\\img\\loc3.jpg')

if img is None:
    print("❌ 無法讀取 loc3.jpg，請確認檔案路徑")
else:
    print(f"圖片尺寸: {img.shape}")
    
    # 轉換為灰階
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 二值化
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 尋找輪廓
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"\n找到 {len(contours)} 個輪廓")
    print("="*80)
    
    # 過濾主要輪廓（最外層且面積較大）
    main_contours = []
    for i, (contour, hier) in enumerate(zip(contours, hierarchy[0])):
        parent_idx = hier[3]
        if parent_idx == -1:  # 最外層輪廓
            area = cv2.contourArea(contour)
            if area > 1000:  # 過濾小面積
                main_contours.append((i, contour, area))
    
    # 按面積排序
    main_contours.sort(key=lambda x: x[2], reverse=True)
    
    # 建立遮罩
    # 遮罩1: 單一輪廓遮罩（最大的物件）
    mask1 = np.zeros(gray.shape, dtype=np.uint8)
    if len(main_contours) > 0:
        cv2.drawContours(mask1, contours, main_contours[0][0], 255, -1)  # -1 表示填滿
    
    # 遮罩2: 所有輪廓遮罩
    mask2 = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask2, contours, -1, 255, -1)
    
    # 使用遮罩提取區域
    result1 = cv2.bitwise_and(img, img, mask=mask1)
    result2 = cv2.bitwise_and(img, img, mask=mask2)
    
    # 反轉遮罩（背景）
    mask1_inv = cv2.bitwise_not(mask1)
    result1_bg = cv2.bitwise_and(img, img, mask=mask1_inv)
    
    # 視覺化 - 3行3列
    fig = plt.figure(figsize=(18, 18))
    
    # 第一行：原始圖片和處理過程
    plt.subplot(3, 3, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Original Image', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 2)
    plt.imshow(gray, cmap='gray')
    plt.title('Grayscale', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 3)
    plt.imshow(binary, cmap='gray')
    plt.title('Binary', fontsize=14)
    plt.axis('off')
    
    # 第二行：輪廓和遮罩
    img_contours = img.copy()
    cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)
    plt.subplot(3, 3, 4)
    plt.imshow(cv2.cvtColor(img_contours, cv2.COLOR_BGR2RGB))
    plt.title(f'All Contours ({len(contours)})', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 5)
    plt.imshow(mask1, cmap='gray')
    plt.title('Mask (Largest Object)', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 6)
    plt.imshow(mask2, cmap='gray')
    plt.title('Mask (All Contours)', fontsize=14)
    plt.axis('off')
    
    # 第三行：遮罩應用結果
    plt.subplot(3, 3, 7)
    plt.imshow(cv2.cvtColor(result1, cv2.COLOR_BGR2RGB))
    plt.title('Result (Largest Object)', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 8)
    plt.imshow(cv2.cvtColor(result2, cv2.COLOR_BGR2RGB))
    plt.title('Result (All Contours)', fontsize=14)
    plt.axis('off')
    
    plt.subplot(3, 3, 9)
    plt.imshow(cv2.cvtColor(result1_bg, cv2.COLOR_BGR2RGB))
    plt.title('Background (Inverted Mask)', fontsize=14)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# 練習15-5 過濾輪廓大小*
import cv2
import numpy as np

# 讀取圖片（灰階即可）
img = cv2.imread(r"C:\img\contours.bmp", 0)

# 二值化
ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# 找輪廓
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print("=== 所有輪廓面積 ===")
areas = []

for i, cnt in enumerate(contours):
    area = cv2.contourArea(cnt)
    areas.append(area)
    print(f"輪廓 {i}: 面積 = {area}")

# 過濾面積大於 15000（黑色背景）
img_area = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

for i, cnt in enumerate(contours):
    if areas[i] > 15000:
        cv2.drawContours(img_area, [cnt], -1, (0, 0, 255), 2)

cv2.imshow("Area > 15000", img_area)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 練習15-6 輪廓長度*
import cv2
import numpy as np

# 讀取圖片
img = cv2.imread(r"C:\img\contours0.bmp", 0)

# 二值化
ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# 找輪廓（使用 RETR_LIST 找到所有輪廓）
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

print("=== 所有輪廓長度 ===")
lengths = []

for i, cnt in enumerate(contours):
    length = cv2.arcLength(cnt, True)
    lengths.append(length)
    print(f"輪廓 {i}: 長度 = {length}")

# 計算平均長度
avg_length = np.mean(lengths)
print(f"\n平均輪廓長度 = {avg_length}")

# 畫出大於平均長度的輪廓（黑色背景）
img_len = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

for i, cnt in enumerate(contours):
    if lengths[i] > avg_length:
        cv2.drawContours(img_len, [cnt], -1, (0, 255, 0), 2)

cv2.imshow("Length > Average", img_len)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 練習15-10 矩形框*
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib 字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 讀取圖片
img = cv2.imread(r"C:\img\cc.bmp")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 二值化
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# 找輪廓
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"找到 {len(contours)} 個輪廓")
print("="*80)

# 方法1: 使用 cv2.drawContours 繪製矩形框
img_method1 = img.copy()

# 方法2: 使用 cv2.rectangle 繪製矩形框
img_method2 = img.copy()

for i, cnt in enumerate(contours):
    # 獲取矩形框的頂點與長寬
    x, y, w, h = cv2.boundingRect(cnt)
    
    print(f"\n輪廓 {i}:")
    print(f"  左上角座標: ({x}, {y})")
    print(f"  寬度: {w}, 高度: {h}")
    print(f"  右下角座標: ({x+w}, {y+h})")
    
    # 方法1: 使用 drawContours
    # 建立矩形的四個頂點座標
    rect_points = np.array([
        [x, y],           # 左上
        [x+w, y],         # 右上
        [x+w, y+h],       # 右下
        [x, y+h]          # 左下
    ], dtype=np.int32)
    
    # drawContours 需要的格式是 [contours]
    cv2.drawContours(img_method1, [rect_points], -1, (0, 255, 0), 2)
    
    # 方法2: 使用 rectangle
    cv2.rectangle(img_method2, (x, y), (x+w, y+h), (255, 0, 0), 2)

print("\n" + "="*80)

# 視覺化比較
fig = plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image', fontsize=14)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(img_method1, cv2.COLOR_BGR2RGB))
plt.title('Method 1: cv2.drawContours()\n綠色矩形框', fontsize=14)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(img_method2, cv2.COLOR_BGR2RGB))
plt.title('Method 2: cv2.rectangle()\n藍色矩形框', fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

# 額外示範：同時顯示輪廓和矩形框
img_combined = img.copy()

for i, cnt in enumerate(contours):
    # 繪製原始輪廓（黃色）
    cv2.drawContours(img_combined, [cnt], -1, (0, 255, 255), 2)
    
    # 繪製矩形框（紅色）
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img_combined, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    # 標註矩形框的頂點
    cv2.circle(img_combined, (x, y), 5, (255, 0, 0), -1)  # 左上角（藍色）
    cv2.circle(img_combined, (x+w, y+h), 5, (0, 255, 0), -1)  # 右下角（綠色）

plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(img_combined, cv2.COLOR_BGR2RGB))
plt.title('Combined: 輪廓(黃) + 矩形框(紅) + 頂點標記', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()


In [2]:
# 練習15-14*
import cv2
import numpy as np

# 1. 讀取圖片
img = cv2.imread(r'C:\img\Chand.jpg')

if img is None:
    raise FileNotFoundError("找不到 Chand.jpg")

# 2. 轉灰階
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 3. 高斯模糊去噪
blur = cv2.GaussianBlur(gray, (5,5), 0)

# 4. 邊緣檢測
edges = cv2.Canny(blur, 50, 150)

# 5. 找輪廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 6. 將所有輪廓點合併，計算最外層凸包
all_points = np.vstack(contours)  # 把所有輪廓點合併
hull = cv2.convexHull(all_points)

# 7. 畫出凸包
cv2.polylines(img, [hull], True, (0, 255, 0), 2)

# 8. 顯示結果
cv2.imshow("Outer Convex Hull", img)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [1]:
# 練習15-16*
import cv2
import numpy as np

# 讀取圖片
img = cv2.imread("C://img/hand.bmp")
if img is None:
    raise FileNotFoundError("找不到圖片，請確認路徑是否正確")

# 影像預處理
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 增加模糊程度有助於簡化輪廓
blur = cv2.GaussianBlur(img_gray, (7, 7), 0)
# 使用 Otsu 自動閾值通常比 Canny 更適合處理手部這類實體物件
_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 找輪廓 (只取外框)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

img_result = img.copy()

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < 500: # 過濾雜訊，手掌面積通常很大
        continue

    # 關鍵：使用 Douglas-Peucker 演算法
    # epsilon 是逼近精度，值越小越貼合（點越多），值越大越簡化（點越少）
    # 通常設定為周長的 1% ~ 5%
    perimeter = cv2.arcLength(cnt, True)
    epsilon = 0.002 * perimeter 
    approx = cv2.approxPolyDP(cnt, epsilon, True)

    # 繪製多邊形 (綠色線)
    cv2.drawContours(img_result, [approx], -1, (0, 255, 0), 2)
    
    # 標註節點 (紅色點)
    for p in approx:
        x, y = p[0]
        # cv2.circle(img_result, (x, y), 5, (0, 0, 255), -1)

    print(f"原本點數: {len(cnt)}, 逼近後點數: {len(approx)}")

# 顯示結果
cv2.imshow("Optimized Approximation", img_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

原本點數: 440, 逼近後點數: 41


In [ ]:
# 練習15-19 點到輪廓的垂直距離
import cv2
import numpy as np

# 讀取圖片
img = cv2.imread("C:\\img\\cs.bmp")
if img is None:
    raise FileNotFoundError("找不到圖片 C:\\img\\cs.bmp")

# 轉灰階
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 二值化
_, binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)

# 找輪廓
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours:
    raise ValueError("未找到任何輪廓")

# 計算最外層凸包
hull = cv2.convexHull(contours[0])

# 測試點
test_point = (300, 150)

# 計算點到輪廓的垂直距離
# True: 計算距離， False: 僅檢查點是否在輪廓內
result = cv2.pointPolygonTest(hull, test_point, True)
result2 = cv2.pointPolygonTest(hull, test_point, False)
# 顯示結果
if result2 < 0:
    print(f"點 {test_point} 在輪廓外部，距離為 {result:.2f}")
elif result2 > 0:
    print(f"點 {test_point} 在輪廓內部，距離為 {result:.2f}")
else:
    print(f"點 {test_point} 在輪廓上")

# 視覺化
img_result = img.copy()
cv2.drawContours(img_result, [hull], -1, (0, 255, 0), 2)  # 繪製凸包
cv2.circle(img_result, test_point, 5, (0, 0, 255), -1)  # 繪製測試點

cv2.imshow("Point to Contour Distance", img_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# 練習15-27 極值*
import cv2
import numpy as np

# 讀圖
img = cv2.imread("C:/img/ct.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 二值化
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# 找輪廓（一定要 TREE）
contours, hierarchy = cv2.findContours(
    thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

mask = np.zeros_like(gray)

# 先找「內圈中面積最大的那一個」
max_inner_area = 0
max_inner_idx = -1

for i, cnt in enumerate(contours):
    parent = hierarchy[0][i][3]
    if parent != -1:  # 內圈
        area = cv2.contourArea(cnt)
        if area > max_inner_area:
            max_inner_area = area
            max_inner_idx = i

# 只畫「最大內圈」
if max_inner_idx != -1:
    cv2.drawContours(mask, [contours[max_inner_idx]], -1, 255, -1)

# 極值
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(gray, mask=mask)
meanVal = cv2.mean(gray, mask=mask)
print("最小值:", minVal, "座標:", minLoc)
print("最大值:", maxVal, "座標:", maxLoc)
print("平均值:", meanVal)

# 顯示
obj = cv2.bitwise_and(img, img, mask=mask)
cv2.imshow("Inner Circle Only", obj)
cv2.imshow("Mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


最小值: 42.0 座標: (87, 90)
最大值: 200.0 座標: (90, 110)
平均值: (85.45594913714804, 0.0, 0.0, 0.0)


In [ ]:
# 練習15-28 平均顏色*

import cv2
import numpy as np


img = cv2.imread("C:/img/ct.png")
if img is None:
    raise FileNotFoundError("找不到圖片")

# 2. 將 BGR 轉換為 BGRA (增加第四個通道 Alpha)
# 此時 Alpha 通道預設全為 255 (完全不透明)
img_bgra = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

# 3. 找輪廓與建立遮罩 (這部分邏輯維持不變)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

mask = np.zeros_like(gray)
max_inner_idx = -1
max_inner_area = 0

if hierarchy is not None:
    for i, cnt in enumerate(contours):
        parent = hierarchy[0][i][3]
        if parent != -1:  # 內圈
            area = cv2.contourArea(cnt)
            if area > max_inner_area:
                max_inner_area = area
                max_inner_idx = i

if max_inner_idx != -1:
    cv2.drawContours(mask, [contours[max_inner_idx]], -1, 255, -1)

    # --- 關鍵步驟：計算 BGRA 平均值 ---
    # 我們將 img_bgra 傳入 cv2.mean，並搭配遮罩
    mean_val = cv2.mean(img_bgra, mask=mask)

    # mean_val 會是一個包含 4 個數值的元組
    print(f"B 通道平均值: {mean_val[0]}")
    print(f"G 通道平均值: {mean_val[1]}")
    print(f"R 通道平均值: {mean_val[2]}")
    # print(f"A 通道平均值: {mean_val[3]:.2f}")  # 這就是 Alpha 通道

    # 4. 顯示結果：將遮罩套用到 BGRA 影像上
    # 這樣背景會變成完全透明 (如果儲存為 PNG 的話)
    res_bgra = cv2.bitwise_and(img_bgra, img_bgra, mask=mask)
    
    cv2.imshow("Inner Object (BGRA)", res_bgra)
    cv2.waitKey(0)
else:
    print("找不到指定的內圈物件")

cv2.destroyAllWindows()

B 通道平均值: 85.45594913714804
G 通道平均值: 85.45594913714804
R 通道平均值: 85.45594913714804


In [ ]:
# 練習15-29 極點*
import cv2
import numpy as np

# 1. 讀圖
img = cv2.imread("C:/img/cs.bmp")
if img is None:
    raise FileNotFoundError("找不到圖片")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 2. 二值化
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# 3. 找輪廓（只要外輪廓即可）
contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

# 4. 取最大輪廓（避免雜訊）
cnt = max(contours, key=cv2.contourArea)

# 5. 找極點
leftmost   = tuple(cnt[cnt[:,:,0].argmin()][0])
rightmost  = tuple(cnt[cnt[:,:,0].argmax()][0])
topmost    = tuple(cnt[cnt[:,:,1].argmin()][0])
bottommost = tuple(cnt[cnt[:,:,1].argmax()][0])

print("最左點:", leftmost)
print("最右點:", rightmost)
print("最上點:", topmost)
print("最下點:", bottommost)

# 6. 畫在圖上
result = img.copy()

cv2.circle(result, leftmost,  5, (255, 0, 0), -1)   # 藍
cv2.circle(result, rightmost, 5, (0, 255, 0), -1)   # 綠
cv2.circle(result, topmost,   5, (0, 0, 255), -1)   # 紅
cv2.circle(result, bottommost,5, (0, 255, 255), -1) # 黃

cv2.imshow("Extreme Points", result)
cv2.waitKey(0)
cv2.destroyAllWindows()


最左點: (np.int32(202), np.int32(135))
最右點: (np.int32(423), np.int32(120))
最上點: (np.int32(369), np.int32(69))
最下點: (np.int32(216), np.int32(179))
